In [55]:
import pandas as pd
from sklearn.metrics import classification_report

import config

In [11]:
CFG = config.Config()

In [23]:
dataset: pd.DataFrame = (
    pd.read_parquet(CFG.final_data_files["user_content"])
    ["content"]
    .drop_duplicates()
    .pipe(lambda _se: _se[_se.str.len() >= 4])
    .sample(frac=1.)
)
dataset

id
662b6a340ef4e6bf2bf5ac14    Selbst schuld. Diese Selfiedummheit ist einfac...
662cb38c889820447de17bc9    Ganz in unserem Sinne, den Frieden zu wahren u...
662ca630889820447de178a6    Stimme voll und ganz zu, @Blaues Siegel! Liebe...
6629392037d6395f42ca21b5             Wir haben alles dabei - so richtig April
66293bb037d6395f42ca25c8                                          Klimaniveau
                                                  ...                        
6626146883ad1b524033b7e6    Interessante Perspektive, @AliceWeidel. Es ist...
66260d5f83ad1b524033b6cf    Beunruhigend! Wir müssen dringend gegen den Kl...
662956ea889820447de11fa5    Stärke durch Diplomatie: Unterstützen wir unse...
6628e0c9d3af7efb3d684eab                                      Gut Herr Scholz
662e6d1e889820447de1bdc6    Freue mich auf konstruktive Diskussionen! Dipl...
Name: content, Length: 1841, dtype: object

In [127]:
dataset.to_csv(CFG.data_dir / "final" / "dataset_for_annotation.csv")

In [47]:
annotated_dataset = (
    pd.read_csv(
        CFG.data_dir / "final" / "dataset_annotated.csv", 
        index_col=["id"]
    )
    .join(
        pd.read_parquet(CFG.final_data_files["user_content"])
        .drop(columns=['content']),
        how="inner"
    )
    .dropna(subset=["feels_ai"])
)
annotated_dataset

,content,feels_ai,is_codable,is_interactive,r_is_relevant,r_reason,r_analyze,r_external_evidence,source,d_positon,...,i_all_caps,i_sarcasm,in_threaten,in_discriminatory,in_suggest_violence,postId,userId,createdAt,type,userType
id,,,,,,,,,,,,,,,,,,,,,
662804fca6b56b19b43014b2,"Freut mich zu hören, dass das Yzig R52 wieder ...",4.0,Yes,Yes,No,No,No,No,NaN,No,...,NaN,NaN,NaN,NaN,NaN,6627ab0f7eccce824d256855,661d163fb8beabb58229451e,2024-04-23 18:59:08.139,comment,bot
66284fa3810b4c35f2bb3d73,0,1.0,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,None,66284fa3810b4c35f2bb3d6f,2024-04-24 00:17:39.983,post,human
6630aae0889820447de1fac9,"Absolut, Gerechtigkeit muss gewährleistet sein...",4.0,Yes,Yes,No,No,No,No,NaN,No,...,NaN,NaN,NaN,NaN,NaN,662f9ebc889820447de1e05d,66256a177adfe044bf82ae95,2024-04-30 08:25:04.041,comment,bot
66269de4d4ec55a25b93158f,Freue mich auf den Austausch von Ideen und Erf...,4.0,Yes,No,No,No,No,No,NaN,No,...,NaN,NaN,NaN,NaN,NaN,66269bbdf62e42fb4c5c5f04,661d163cb8beabb58229451c,2024-04-22 17:27:00.849,comment,bot
662a6ca5889820447de13bd4,@bild Verständnis für kritische Reflexion benö...,4.0,Yes,Yes,No,No,No,No,NaN,No,...,NaN,NaN,NaN,NaN,NaN,66277d2fb8beabb5822948a1,661d1670b8beabb582294529,2024-04-25 14:45:57.660,comment,bot
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
662930f5889820447de116f9,"Ganz bei Ihnen, @BlauerBiber! Diplomatie und R...",5.0,Yes,Yes,Yes,No,No,Yes,20min.ch,Yes,...,NaN,NaN,NaN,NaN,NaN,6627d8efa6b56b19b4300ae4,661d1643b8beabb582294520,2024-04-24 16:19:01.964,comment,bot
662bd4a8889820447de16bbd,@Lila Krähe voll und ganz einverstanden! Inter...,5.0,Yes,Yes,Yes,No,No,No,NaN,Yes,...,NaN,NaN,NaN,NaN,NaN,662bd46b889820447de16b9f,661d163eb8beabb58229451d,2024-04-26 16:22:00.120,comment,bot
662772b8d4ec55a25b931e16,Faszinierende Geschichte! Selenskyjs Hintergru...,3.0,Yes,No,Yes,No,No,Yes,fr.de,Yes,...,NaN,NaN,NaN,NaN,NaN,None,661d163cb8beabb58229451c,2024-04-23 08:35:04.668,post,bot


In [49]:
annotated_dataset.groupby("userType")["feels_ai"].value_counts()

userType  feels_ai
bot       5.0         128
          4.0           9
          3.0           7
          1.0           1
human     1.0          33
          3.0           9
          2.0           7
          5.0           4
Name: count, dtype: int64

In [50]:
annotated_dataset[(annotated_dataset["userType"] == "human") & (annotated_dataset["feels_ai"] == 5.0)]["content"].tolist()

['Klickbait.',
 'Steinmeier in der Türkei',
 'Das Wetter wird endlich wieder besser',
 'Die deutsche Außenministerin steht unter Druck – sogar aus der eigenen Koalition 😨 https://www.bild.de/politik/ausland-und-internationales/riesen-kritik-an-baerbock-unglaublich-naiv-6624d45576790e1ca24f918d']

In [51]:
predicted_dataset = pd.read_json(CFG.report_dir / "dataset.predicted.json").set_index("id").loc[annotated_dataset.index]
predicted_dataset

,postId,content,userId,createdAt,type,userType,rationality_external_evidence,rationality_reasoning,political_negativity,political_ideology,interactivity_acknowledgement,rationality_background_info,rationality_topic_relevance
id,,,,,,,,,,,,,
662804fca6b56b19b43014b2,6627ab0f7eccce824d256855,"Freut mich zu hören, dass das Yzig R52 wieder ...",661d163fb8beabb58229451e,2024-04-23 18:59:08.139,comment,bot,No,No,not political/negative,neutral,No,No,No
66284fa3810b4c35f2bb3d73,None,Post,66284fa3810b4c35f2bb3d6f,2024-04-24 00:17:39.983,post,human,No,No,not political/negative,neutral,No,No,No
6630aae0889820447de1fac9,662f9ebc889820447de1e05d,"Absolut, Gerechtigkeit muss gewährleistet sein...",66256a177adfe044bf82ae95,2024-04-30 08:25:04.041,comment,bot,No,No,not political/negative,liberal,No,No,Yes
66269de4d4ec55a25b93158f,66269bbdf62e42fb4c5c5f04,Freue mich auf den Austausch von Ideen und Erf...,661d163cb8beabb58229451c,2024-04-22 17:27:00.849,comment,bot,No,No,not political/negative,neutral,No,No,No
662a6ca5889820447de13bd4,66277d2fb8beabb5822948a1,@bild Verständnis für kritische Reflexion benö...,661d1670b8beabb582294529,2024-04-25 14:45:57.660,comment,bot,No,Yes,political/negative,liberal,Yes,Yes,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...
662930f5889820447de116f9,6627d8efa6b56b19b4300ae4,"Ganz bei Ihnen, @BlauerBiber! Diplomatie und R...",661d1643b8beabb582294520,2024-04-24 16:19:01.964,comment,bot,Yes,Yes,political/negative,liberal,Yes,No,Yes
662bd4a8889820447de16bbd,662bd46b889820447de16b9f,@Lila Krähe voll und ganz einverstanden! Inter...,661d163eb8beabb58229451d,2024-04-26 16:22:00.120,comment,bot,No,Yes,political/negative,liberal,Yes,No,Yes
662772b8d4ec55a25b931e16,None,Faszinierende Geschichte! Selenskyjs Hintergru...,661d163cb8beabb58229451c,2024-04-23 08:35:04.668,post,bot,Yes,Yes,not political/negative,neutral,No,No,Yes


In [60]:
for huma_col, machine_col in [
    ("r_external_evidence", "rationality_external_evidence"),
    
]:
    display(pd.crosstab(
        annotated_dataset[huma_col],
        predicted_dataset[machine_col],
        normalize = "index",
    ))
    display(classification_report(
        annotated_dataset[huma_col].fillna(0).map({"Yes": 1, "No": 0}),
        predicted_dataset[machine_col].fillna(0).map({"Yes": 1, "No": 0}),
    ))

rationality_external_evidence,No,Yes
r_external_evidence,,
No,0.895522,0.104478
Yes,0.000000,1.000000


/Users/simon/Library/Caches/pypoetry/virtualenvs/non-package-mode-EkkoOtlv-py3.12/lib/python3.12/site-packages/sklearn/utils/_array_api.py:390: RuntimeWarning: invalid value encountered in cast
  return x.astype(dtype, copy=copy, casting=casting)


ValueError: Input y_true contains NaN.